# These research-papers were used for this code:

* SnapMix: Semantically Proportional Mixing for Augmenting Fine-grained Data - http://arxiv.org/abs/2012.04846

And to understand the SnapMix article:
* Mixup: Beyond Empirical Risk Minimization - http://arxiv.org/abs/1710.09412
* Learning Deep Features for Discriminative Localization - http://arxiv.org/abs/1512.04150

In [ ]:
# directories and paths to data:

import torch
import os
import pandas as pd
from pathlib import Path
import numpy as np

base_path = Path('../input/cassava-leaf-disease-classification')
train_img_dir = os.path.join(base_path,'train_images')
test_img_dir = os.path.join(base_path,'test_images')

train_images = os.listdir(train_img_dir)
test_images = os.listdir(test_img_dir)

train_df = pd.read_csv(os.path.join(base_path,'train.csv'))

diseaseMapping = pd.read_json(os.path.join(base_path,'label_num_to_disease_map.json'), typ='series')

In [ ]:
# test if training data have been loaded:
# train_df.shape

# **Plan:**
- look up necessary preprocessing for resnet50 input: implement it:  ok
- load a batch of 2 images to test the snapmix augmentation part: ok

When that is ok:
- write data-loader to read in data and label: ok
- test for snapmix batch-loss-function: ok
- write training-loop: ok
- add classifier to network: ok
- freeze extractor, train classifier: ok
- implemented in pytorch save & load best performing model: ok
- thaw last conv layer, train some more: ok - thawing last conv layer after 5 epochs.
- submit code to Kaggle:
- if better than 44% - upload to github:
- finally: add midlayer information like described in the paper
- if that is hopefully even better - upload to github:

# Data splitting, data-sets, data-loader here:

In [ ]:
# use this to split data in train_df into TRAINING AND EVALUATION (but not an additional test set):
fraction_training = 0.8

assert fraction_training < 1, "!fraction_training must be smaler then 1, else there will be no evaluation data!"

eval_df = train_df.sample(frac=1-fraction_training)
train_df = train_df.drop(eval_df.index)


In [ ]:
# test/ controll for data splitting:

#eval_df.shape, eval_df.shape[0]/len(index), train_df.shape, train_df.shape[0]/len(index)

In [ ]:
# use this to split data in train_df into TRAINING, EVALUATION AND TEST SET:

# choose splitting fractions:
#fraction_training = 0.8
#fraction_evaluation = 0.1
#fraction_test = 0.1

#assert fraction_training + fraction_evaluation + fraction_test == 1, "fraction_training + fraction_evaluation + fraction_test must sum to 1"

#f = 1 - fraction_training
#eval_df = train_df.sample(frac = f)
#train_df = train_df.drop(eval_df.index)
#test_df = eval_df.sample(frac= fraction_test/f)
#eval_df = eval_df.drop(test_df.index)

#eval_df.shape, test_df.shape, train_df.shape

In [ ]:
import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset

class CassavaImageDataset(Dataset):
    """
    Reads image name and label from pandas data-frame, loads data via PIL, applies transform to image and label and return (image, label) pair.
    
    Args:
        label_image_dataframe (pandas datafram): pandas data-frame containing image name and label
        img_dir_path (string): path to the directory containing the images
        transform: transform to be applied to the data/images
        label_transform: transform to be applied to the labels
    
    Returns:
        sample (dictionary): {"image": image, "label": label, "img_name" : image_name}
    """
    def __init__(self, image_label_dataframe, img_dir_path, transform=None, label_transform=None):
        self.image_label_df = image_label_dataframe
        self.img_dir_path = img_dir_path
        self.transform = transform
        self.label_transform = label_transform

    def __len__(self):
        return len(self.image_label_df)

    def __getitem__(self, idx):
        image_name = self.image_label_df.iloc[idx, 0]
        img_path = os.path.join(self.img_dir_path, image_name)
        image = Image.open(img_path)
        label = self.image_label_df.iloc[idx, 1]
        if self.transform is not None:
            image = self.transform(image)
        if self.label_transform is not None:
            label = self.label_transform(label)
        sample = {"image": image, "label": label, "img_name" : image_name}
        
        return sample

## Construction of datasets and dataloaders

In [ ]:
# construction of datasets and dataloader:

from torchvision import transforms

# necessary resnet50 preprocessor:
preprocess_resnet50 = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[4.485, 0.456, 0.406],
        std= [0.229, 0.224, 0.225]),
])

from torch.utils.data import DataLoader

# definition of dataloaders:
train_ds = CassavaImageDataset(train_df, train_img_dir, transform=preprocess_resnet50)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)

eval_ds = CassavaImageDataset(eval_df, train_img_dir, transform=preprocess_resnet50)
eval_size = eval_df.shape[0]
eval_dl = DataLoader(eval_ds, batch_size=32, shuffle=False)

# check if test_df has been created/ is desired - if so, create the dataloader test_dl for it:
if "test_df" in locals():
    test_ds = CassavaImageDataset(test_df, train_img_dir, transform=preprocess_resnet50)
    test_size = test_df.shape[0]
    test_dl = DataLoader(test_ds, batch_size=test_size, shuffle=False)
    print("dataloader test_dl created.")


In [ ]:
# test dataloader:

#for i, sample in enumerate(train_dl):
#    if i > 1:
#       break
#    print(sample["image"].shape)
#    print(sample["label"].shape)

# Model definition: resnet50, featureNet, cassava-classifier here:

In [ ]:
# DEFINITION OF THE MODEL(S):

import torch.nn as nn
import torchvision.models as models

class Cassava_resnet50(nn.Module):
    def __init__(self, freeze_featureNet=True):
        """
            freeze_featureNet (bool): if true sets requires_grad = False for all layers except the fc classifier layer.
        """
        super(Cassava_resnet50, self).__init__()
        #resnet50 = models.resnet50(pretrained=True) # needs internet connection
        #--- without internet connection:
        resnet50 = models.resnet50(pretrained=False)
        dict_resnet50 = torch.load("../input/resnet50/resnet50.pth")
        resnet50.load_state_dict(dict_resnet50)
        #---
        resnet_no_classifier = list(resnet50.children())[:-2]
        self.featureNet = nn.Sequential(*resnet_no_classifier)
        self.avgPool = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.cassava_fc = nn.Linear(2048, 5, bias=True)
        
        if freeze_featureNet:
            for param in self.featureNet.parameters():
                param.requires_grad = False
        
        
    def forward(self, x):
        """
        Returns: 
            two outputs: feature_model_output, classification_model_output
        """
        x = self.featureNet(x)
        y = self.avgPool(x)
        y = torch.squeeze(y)
        y = self.cassava_fc(y)
        
        return x,y



In [ ]:
# test model creation:

#model = Cassava_resnet50()
#print(model.cassava_fc.weight)

# Functions for snapmix here:

In [ ]:
# functions for snap-mix augmentation and loss:

import torchvision.transforms as T
import torchvision.transforms.functional as TF
import numpy as np


def snapmix_batch_loss(label_batch, y_scores, box_weights1 = None, box_weights2 = None):
    """
    Calculates the loss according to snap-mix algorithm.
    
    Args:
        label_batch : true labels
        y_scores : raw-score vectors for label-prediction
        box_weights1 : semantic box weights of patched-into images
        box_weights2 : semantic box weights of patched-in images
    
    Returns:
        snap-mix loss
    """
    loss = torch.nn.CrossEntropyLoss()(y_scores, label_batch)
    return torch.mean(torch.mul(loss, (1 - box_weights1 + box_weights2)), dim=0)


def snapmix_batch_augmentation(model, img_batch, label_batch, alpha=0.2):
    """
    Applies, the SnapMix-augmentation to the images and labels within a data batch with respect to a model.

    Args:
        model (Cassava_resnet50) : the part of the model ending in the last feature map (convolution) of the resnet50
        img_batch (torch.tensor) : batch with images, all the same shape
        label_batch (numpy list) : batch with labels for the images
        alpha (float), optional: parameter for beta-distribution generating image shrinking-factor for box-area

    Returns:
        augmented_images : the augmented input-images
        label_batch2 : the labels of the images that have been patched into the input-images
        box_weights1 : batch of semantic weights of cut-out-boxes
        box_weights2 : batch of semantic weights of patched-in-boxes
    """
    # pytorch uses: B x C x H x W:
    input_batch_size = img_batch.shape[0]
    input_img_height = img_batch.shape[2]
    input_img_width = img_batch.shape[3]
        
    box1 = random_box(input_img_width, input_img_height, alpha=alpha)
    box2 = random_box(input_img_width, input_img_height, alpha=alpha)
    
    # To increase speed we copy and permutate the images of the batch and patch the images from this
    # new batch - so we have allready the semantic percentage map for the copied batch:

    # build another image batch from the input batch:
    permutation = torch.randperm(input_batch_size)
    label_batch = label_batch.type(torch.int64)
    img_batch2 = torch.clone(img_batch.detach())
    img_batch2 = img_batch2[permutation]

    # get spm and calculate boxweights:
    SPM1 = batch_semantic_percentage_map(
        model = model,
        img_batch = img_batch,
        label_batch = label_batch,
    )
    
    # copy and permute the semantic percentage maps of the first batch in the same way as the
    # images of img_batch2 :
    #SPM2 = torch.clone(SPM1)
    #SPM2 = SPM2[permutation, :, :]
    
    # crop boxes:
    x11, y11, x12, y12 = box1
    x21, y21, x22, y22 = box2
    height_box1 = x12 - x11
    width_box1 = y12 - y11
    height_box2 = x22 - x21
    width_box2 = y22 - y21
    
    cropped_SPM1 = TF.crop(SPM1, top=x11, left=y11, height=height_box1, width=width_box1)
    box_weights1 = torch.sum(cropped_SPM1, dim=(1, 2))
    #cropped_SPM2 = TF.crop(SPM2, top=x21, left=y21, height=height_box2, width=width_box2)
    #box_weights2 = torch.sum(cropped_SPM2, dim=(1, 2))
    #print("box_weights2 :", box_weights2)
    cropped_SPM2 = TF.crop(SPM1, top=x21, left=y21, height=height_box2, width=width_box2)
    box_weights2 = torch.sum(cropped_SPM2, dim=(1, 2))
    box_weights2 = box_weights2[permutation]
    #print("box_weights12:", box_weights12[permutation])

    # fix for cases where box_weights are not well defined: we take the relative areas of the boxes:
    rel_area1 = height_box1 * width_box1 /  (input_img_width * input_img_height)
    rel_area2 = height_box2 * width_box2 / (input_img_width * input_img_height)
    box_weights1[torch.isnan(box_weights1)] = rel_area1
    box_weights2[torch.isnan(box_weights2)] = rel_area2

    #crop and paste images:
    cropped_img2s = TF.crop(img_batch2, top=x11, left=y11, height=height_box1, width=width_box1)
    resized_cropped_img2s = T.Resize((height_box1, width_box1))(cropped_img2s)
    img_batch[:, :, x11: x12, y11:y12] = resized_cropped_img2s

    return img_batch, box_weights1, box_weights2


def batch_semantic_percentage_map(model, img_batch, label_batch):
    """
    Calculates the SPM - Semantic Percentage Map of a batch of images.

    Args:
        model (Cassava_resnet50): 
        img_batch: batch of input images
        label_batch: batch of the images labels

    Returns:
        the SPMs (Semantic Percentage Maps) for a batch of images.
    """
    # weights for determining the contribution to the final classification:
    # classing_weights.shape = [number of classes, number of fc-layer neurons], i.e. here [5,2048]
    classing_weights = model.cassava_fc.weight
    # the batch of all feature map batches for all images in the img_batch:
    feature_maps_batch, _ = model(img_batch) 

    # Calculate Class Activation Map (CAM):
    # for the numbers: feature_maps_batch.shape = [number of images, channels, height, width]
    img_batch_size = feature_maps_batch.shape[0] 
    feature_map_height = feature_maps_batch.shape[2]
    feature_map_width = feature_maps_batch.shape[3]
    CAM_batch = torch.zeros((img_batch_size, feature_map_width, feature_map_height))

    clw_batch_matrix = classing_weights[label_batch, :]
    for i in range(img_batch_size):
        class_weights = clw_batch_matrix[i,:].detach()
        feature_map = feature_maps_batch[i,:,:,:].detach()
        CAM_batch[i] = torch.tensordot(class_weights, feature_map, dims=1)
        
    # upsampling feature map to size of image:
    image_width = img_batch.shape[-1]
    image_height = img_batch.shape[-2]
    resized_CAM_batch = T.Resize((image_height, image_width))(CAM_batch)
    
    # move minimal value in tensor to zero, to avoid extinction when summing over the tensor:
    resized_CAM_batch -= torch.min(resized_CAM_batch)
    normalization_factor = torch.sum(resized_CAM_batch) + 1e-8
    resized_CAM_batch /= normalization_factor

    return resized_CAM_batch


def random_box(im_width, im_height, alpha, minimal_width=3, minimal_height=3):
    """
    Returns a random box=(x1, y1, x2, y2) with 
    0 < x1, x2 < im_width
    and 
    0< y1, y2, < im_height 
    that spans an area such that:
    lambda_img = (x2 - x1) * (y2 - y1) / (im_width * im_height), 
    where lambda_img is randomly drawn from a beta-distribution beta(alpha, alpha)
    """
    random_width = im_width + 1
    random_height = 0
    
    while random_width > im_width or \
    random_height > im_height or \
    random_height < minimal_height or \
    random_width < minimal_width:
        lambda_img = torch.distributions.beta.Beta(torch.tensor([alpha]), torch.tensor([alpha])).sample().item()
        if lambda_img < 1:
            low = int(torch.maximum(torch.tensor(lambda_img * im_height), torch.tensor(minimal_height)))
            high = torch.minimum(torch.tensor(lambda_img * im_width * im_height/ minimal_width), torch.tensor(im_height))
            high = int(torch.maximum(high, torch.tensor(minimal_height)))
            if low > high:
                print("false low", low)
                print("false high", high)
                print("lambda_img", lambda_img)
                raise ValueError
            elif low + 1 < high - 1:
                random_height = torch.randint(low + 1, high - 1, (1,1)).item()
            elif low + 1 >= high -1:
                random_height = im_height
                random_width = im_width
            random_width = int(torch.floor(torch.tensor(lambda_img * im_width * im_height / random_height)))

    left_upper_x = torch.randint(0, im_width - random_width + 1, (1,1)).item()
    left_upper_y = torch.randint(0, im_height - random_height + 1, (1,1)).item()
    box = (left_upper_x,
           left_upper_y,
           left_upper_x + random_width - 1,
           left_upper_y + random_height - 1)

    return box


In [ ]:
# test : does TF.crop copy or just create a view?
# answer: new object at different place in storage
#import torchvision.transforms.functional as TF

#x11 = 2
#y11 = 2
#height_box1 = 5
#width_box1 = 5
#features = np.zeros([32,3,224,224])
#for data in train_dl:
#        features = data["image"]
#        break
#cropped_features1 = TF.crop(features, top=x11, left=y11, height=height_box1, width=width_box1)
#cropped_features2 = TF.crop(features, top=x11, left=y11, height=height_box1, width=width_box1)

#print(cropped_features2 is cropped_features1) # not the same object
#print(id(features), id(cropped_features1), id(cropped_features2)) # different places in storage

In [ ]:
# tests for function "random_box":
#import torch
#import time

#im_width = 224
#im_height = 224
#alpha = 2.

#im_width = 224
#im_height = 224
#alpha = 1.

#im_width = 224
#im_height = 224
#alpha = 5.

#im_width = 224
#im_height = 512
#alpha = 2.

#im_width = 1024
#im_height = 224
#alpha = 2.

#start_time = time.process_time()
#for i in range(1000):
#    b = random_box(im_width, im_height, alpha, minimal_width=3, minimal_height=3)
#elapsed_time = time.process_time() -start_time
#print("elapsed_time:", elapsed_time)        

# Code for training and evaluation

In [ ]:
import torch.nn as nn


def simple_train_model(model, augmentation_transform, optimizer, inputs, labels):
    """
    Per batch training - no snapmix augmentation.
    """
    optimizer.zero_grad()
    _, y_scores = model(augmentation_transform(inputs))
    loss = nn.CrossEntropyLoss()(y_scores, labels)
    loss.backward()
    optimizer.step()

    # give some feedback how it is going:
    return loss.item()
    
    
def snapmix_train_model(model, optimizer, inputs, labels, alpha = 3.):
    """
    Per batch training - with snapmix augmentation.
    Uses GPU if possible
    """
    #use GPU if possible
    device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))

    with torch.no_grad():
        img_batch, box_weights1, box_weights2 = snapmix_batch_augmentation(model, inputs, labels, alpha=alpha)

    img_batch = img_batch.to(device=device)
    box_weights1 = box_weights1.to(device=device)
    box_weights2 = box_weights2.to(device=device)
    optimizer.zero_grad()
    _, y_scores = model(img_batch) # y_scores: predicted y's in raw-score/logit-form
    loss = snapmix_batch_loss(labels, y_scores, box_weights1 = box_weights1, box_weights2 = box_weights2)
    loss.backward()
    optimizer.step
    
    # give some feedback how it is going:
    return loss.item()
        

def predict(y_scores):
    if len(y_scores.shape)==1:
        y_scores = torch.unsqueeze(y_scores,dim=0)
    probabs = nn.Softmax(dim=1)
    return torch.argmax(probabs(y_scores), dim=1)
    
    

## Training and evaluation loop here

In [ ]:
# THIS IS THE TRAINING- AND EVALUATION LOOP:

import torch
import torch.nn as nn
from torchvision import transforms as T
import time


# use GPU if available:
device = (torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu"))
print(f"Training on device {device}.")

# parameters:

# number of epochs to train - takes approx. 5 mins per epoch:
epochs = 20

# number of epochs after which to thaw the extractor (applies only for newly build model, not when best model is loaded):
thaw_after_epochs = 3

# probability if snapmix augmentation should be applied:
snapmix_probab = 0.5 # value according to Huang et al. 2020 

# alpha parameter for beta-distribution in snap-mix augmentation:
smix_alpha = 5. # value according to Huang et al. 2020 

# Build model or load existing former "best model":
load_best_model = True

# load best model if it exists - unfreeze feature-extractor:
if load_best_model and os.path.exists(os.path.join("./", "best_metric_model.pth")):
    model = Cassava_resnet50()
    best_dict = torch.load(os.path.join("./", "best_metric_model.pth"))
    model.load_state_dict(best_dict)
    for params in model.featureNet.parameters():
        param.requires_grad = False
    model.to(device=device)
    # best model is already pretrained so training of the thawed-layers
    # can continue:
    thaw_after_epochs = 0
    print("Loaded best previous model.")
else:
    model = Cassava_resnet50(freeze_featureNet = True).to(device=device) 
    print("New model build.")

# initialize the optimizer - after moving the model to the device:
optimizer= torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # do this after moving to GPU


# finally - here comes the training-loop:
accuracies = []
best_acc = -1
losses = []
augmentation = T.Compose([
        T.RandomVerticalFlip(p=0.5),
        T.RandomAffine(30, translate=[.2,.2], scale=None, shear=[-10,10,-10,10]),
])

for ep in range(epochs):
    # thaw last conv layer of featureNet after 5 epochs:
    if ep == thaw_after_epochs:
        print("Thawing feature extractor.")
        for param in model.featureNet[7].parameters():
            param.requires_grad = True
    
    start_time = time.process_time()
    print("--- Epoch: {} ---".format(ep))
    loss_train = 0.0
    loss = 0.0
    samples_count = 0
    model.train()
    for i, batch in enumerate(train_dl):
        inputs = batch["image"].to(device=device)
        labels = batch["label"].to(device=device)
        samples_count += len(labels)
        s = np.random.uniform(0,1)
        if s <= snapmix_probab:
            loss = snapmix_train_model(model, optimizer, inputs, labels, alpha = smix_alpha)
        else:
            loss = simple_train_model(model, augmentation, optimizer, inputs, labels)
        loss_train += loss
        # give some feedback to the user:
        if i > 0 and i%100 == 0:
            print(f"- Batch {i} -")
            print(f"- avg. loss: {loss_train / samples_count}")
    
    model.eval()
    print(f"-- Evaluation Epoch {ep} Started --")
    for i, batch in enumerate(eval_dl):
        inputs = batch["image"].to(device=device)
        labels = batch["label"].to(device=device)
        loss = 0.
        _, y_scores = model(inputs)
        preds = predict(y_scores)
        accuracy = torch.sum(preds == labels)/ len(labels)
        accuracies.append(accuracy.item())
        loss = nn.CrossEntropyLoss()(y_scores, labels)
        losses.append(loss.item())
    
    mean_acc = np.mean(accuracies)
    mean_loss = np.mean(losses)
    elapsed_time = time.process_time() - start_time 
    print("Epoch mean accuracy: {0} --- mean loss: {1}".format(mean_acc, mean_loss))
    print(f"Epoch elapsed time: {elapsed_time} ")
    # save the best model:
    if best_acc < mean_acc:
        best_acc = mean_acc
        best_at_epoch = ep
        torch.save(model.state_dict(), os.path.join("./", "best_metric_model.pth"))
        print("model saved as best metric model")

print("Training completed.")
print(f"best accuracy: {best_acc}")
print(f"best model saved at epoch: {best_at_epoch}")


# Code for Kaggle Submission

In [ ]:
# code for kaggle submission - does NOT use CassavaTestDataset (...seemed to work without in the test...):

import pandas as pd
from pathlib import Path
import os
from torch.utils.data import DataLoader

# load best performing model:
output_base_path = Path("./") # best model resides in my output-path
best_model = Cassava_resnet50()
dict_best_model = torch.load(os.path.join(output_base_path, "best_metric_model.pth"))
best_model.load_state_dict(dict_best_model)
best_model.to(device=device)
model.eval()

#load test data:
test_base_path = Path('../input/cassava-leaf-disease-classification') # the assumed Kaggle autograder path to image directory and csv file
test_directory = os.path.join(test_base_path,'test_images') # the assumed Kaggle autograder test images directory
test_df = pd.read_csv(os.path.join(test_base_path, "sample_submission.csv")) # the assumed Kaggle-autograder csv file

# define dataloader for submission: 
test_ds = CassavaImageDataset(test_df, test_directory, transform=preprocess_resnet50)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=False)

all_img_names = []
all_label_predictions = []
for i, batch in enumerate(test_dl):
    inputs = batch["image"].to(device=device)
    _, y_scores = model(inputs)
    predictions = predict(y_scores)
    all_label_predictions.extend(predictions.to(device="cpu").numpy()) # convert to numpy to get the numbers in the batch-tensor
    all_img_names.extend(batch["img_name"])
    
submission_df=pd.DataFrame({"image_id":all_img_names, "label":all_label_predictions})
submission_df.to_csv("submission.csv",index=False) # use index=False to prevend pandas from adding an additional index-column

## CassavaTestDataset - not used

In [ ]:
# Definition of CassavaTestDataset
# might be necessary since the CassavaImageDataset tries to return the label of an image - which is not possible for test-data

#import os
#import pandas as pd
#from PIL import Image
#from torch.utils.data import Dataset

#class CassavaTestDataset(Dataset):
#    """
#    Reads image name and label from pandas data-frame, loads data via PIL, applies transform to image and label and return (image, label) pair.
#    
#    Args:
#        label_image_dataframe (pandas datafram): pandas data-frame containing image name and label
#        img_dir_path (string): path to the directory containing the images
#        transform: transform to be applied to the data/images
#        label_transform: transform to be applied to the labels
    
#    Returns:
#        sample (dictionary): {"image": image, "label": label, "img_name" : image_name}
#    """
#    def __init__(self, image_label_dataframe, img_dir_path, transform=None, label_transform=None):
#        self.image_label_df = image_label_dataframe
#        self.img_dir_path = img_dir_path
#        self.transform = transform
#        self.label_transform = label_transform

#    def __len__(self):
#        return len(self.image_label_df)

#    def __getitem__(self, idx):
#        image_name = self.image_label_df.iloc[idx, 0]
#        img_path = os.path.join(self.img_dir_path, image_name)
#        image = Image.open(img_path)
#        if self.transform is not None:
#            image = self.transform(image)
#        if self.label_transform is not None:
#            label = self.label_transform(label)
#        sample = {"image": image, "img_name" : image_name}
        
#        return sample

In [ ]:
# test for kaggle submission code:
# read the submission.csv:
#sub_df = pd.read_csv(os.path.join(output_base_path, "submission.csv"))
# look at it's content:
#sub_df.head()

 Test Snapmix Function